In [1]:
import os
import numpy as np
from matplotlib.lines import Line2D
from scipy.stats import pearsonr, gaussian_kde, linregress, ttest_ind, sem, zscore
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import norm
from scipy.stats import percentileofscore
from sklearn.utils.validation import check_random_state
from math import factorial
from more_itertools import distinct_permutations
import statsmodels.api as sm

import matplotlib.pyplot as plt

import pandas as pd
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

import numpy as np
from scipy.stats import linregress
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold, ParameterGrid, train_test_split
from tqdm.notebook import tqdm

import warnings
import random
#from torch.utils.data import SubsetRandomSampler
from sklearn.utils import resample


warnings.filterwarnings("ignore")

#import pingouin as pg
from sklearn.utils import resample

In [2]:
from statannot import add_stat_annotation
import seaborn as sns


In [3]:
import numpy as np
from scipy.stats import percentileofscore
from sklearn.utils import check_random_state
from itertools import islice

def permtest(x, y, statistic="mean", max_samples=5000, random_state=42):
    """
    Conducts a permutation test between two independent samples.
    Parameters
    ----------
    x : ndarray
        First set of datapoints.
    y : ndarray
        Second set of datapoints.
    statistic : str or callable, optional
        Function that takes in samples x and y and reports
        statistic of interest. By default, "mean" reports
        the difference of sample means. List of default
        options are ("mean", "median").
    max_samples : int, optional.
        Maximum number of label permutations to try.
    random_state : np.random.RandomState, int, or None.
        If specified, used to seed the random number generator to shuffle the ordering of the datapoints.
    """

    # Initialize random state and function of interest
    rs = check_random_state(random_state)
    stat_func = _get_stat_func(statistic)

    # Concatenate samples
    xy = np.concatenate((x, y))
    n_x = x.size

    # Observed statistic
    observed_stat = stat_func(x, y)

    # Set the number of permutations to compute
    n_perms = max(max_samples, 100000)  # Limit the number of permutations to a manageable number (e.g., 10000)

    # Allocate space for computed statistics
    shuffled_stats = np.empty(n_perms)

    # Print coverage
    print(f"Computing permutations...")

    # Sampling random permutations
    for i in range(n_perms):
        rs.shuffle(xy)
        x_ = xy[:n_x]
        y_ = xy[n_x:]
        shuffled_stats[i] = stat_func(x_, y_)

    # Compute a two-sided p-value. We take the smallest
    # percentile and then multiply by two.
    pval = 2 * min(
        percentileofscore(shuffled_stats, observed_stat, kind="rank") / 100,
        1 - percentileofscore(shuffled_stats, observed_stat, kind="rank") / 100
    )

    return pval

def _get_stat_func(name_or_func):
    """
    Instantiates functions that compute default statistics of interest.
    """
    # If specified function is callable
    if not isinstance(name_or_func, str):
        if callable(name_or_func):
            return name_or_func
        else:
            raise ValueError(
                "`statistic` should be a string like ('mean', 'median')"
                " or a function that takes in samples x, y and returns"
                " the statistic of interest."
            )

    # Default functions
    if name_or_func == "mean":
        return lambda x, y: np.mean(x) - np.mean(y)
    elif name_or_func == "median":
        return lambda x, y: np.median(x) - np.median(y)
    else:
        raise ValueError("Did not recognize statistic.")



In [4]:
import numpy as np

def pooled(x, y, func):
    nx = len(x)
    ny = len(y)
    return np.sqrt(((nx - 1) * func(x) ** 2 + (ny - 1) * func(y) ** 2) / (nx + ny - 2))

def hdmedian(x):
    return np.median(x)

def hdmad(x):
    return 1.4826 * hdmedian(np.abs(x - hdmedian(x)))

def phdmad(x, y):
    return pooled(x, y, hdmad)

def gamma_effect_size(x, y, prob):
    hdquantile_x = np.percentile(x, prob * 100)
    hdquantile_y = np.percentile(y, prob * 100)
    return (hdquantile_y - hdquantile_x) / phdmad(x, y)



In [5]:
def remove_outliers_iqr_01(data):

    data = np.array(data)

    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1


    ir_mult = 0.1
    lower_bound = Q1 - ir_mult * IQR
    upper_bound = Q3 + ir_mult * IQR

    filtered_data = data[(data >= lower_bound) & (data <= upper_bound)]
    
    return filtered_data

def remove_outliers_iqr_05(data):

    data = np.array(data)

    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1


    ir_mult = 0.5
    lower_bound = Q1 - ir_mult * IQR
    upper_bound = Q3 + ir_mult * IQR

    filtered_data = data[(data >= lower_bound) & (data <= upper_bound)]
    
    return filtered_data

def remove_outliers_iqr_10(data):

    data = np.array(data)

    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1

    ir_mult = 1.0
    lower_bound = Q1 - ir_mult * IQR
    upper_bound = Q3 + ir_mult * IQR

    filtered_data = data[(data >= lower_bound) & (data <= upper_bound)]
    
    return filtered_data

def cliff_delta(group1, group2):

    group1_ = remove_outliers_iqr_01(group1)
    group2_ = remove_outliers_iqr_01(group2)
    
    sub_sample_size = np.min([int(group1_.shape[0]*0.7), int(group2_.shape[0]*0.7)])

    delta_l = []
    for i in range(10):
       
        group1 = resample(group1_, n_samples=sub_sample_size, replace=False, random_state=i)
        group2 = resample(group2_, n_samples=sub_sample_size, replace=False, random_state=i)
    
        group1_exp = group1[:, np.newaxis]  
        group2_exp = group2[np.newaxis, :]  
    
        greater = np.sum(group1_exp > group2_exp)
        less = np.sum(group1_exp < group2_exp)
    
        n1, n2 = len(group1), len(group2)
        delta = (greater - less) / (n1 * n2)

    delta_l.append(delta)

    print('N', n1+n2, n1, n2)

    return (np.round(np.mean(delta_l), 4), np.round(np.std(delta_l)*2.58,100))

def cliff_delta05(group1, group2):

    group1_ = remove_outliers_iqr_05(group1)
    group2_ = remove_outliers_iqr_05(group2)
    
    sub_sample_size = np.min([int(group1_.shape[0]*0.7), int(group2_.shape[0]*0.7)])

    delta_l = []
    for i in range(10):
       
        group1 = resample(group1_, n_samples=sub_sample_size, replace=False, random_state=i)
        group2 = resample(group2_, n_samples=sub_sample_size, replace=False, random_state=i)
    
        group1_exp = group1[:, np.newaxis]  
        group2_exp = group2[np.newaxis, :] 
    
        greater = np.sum(group1_exp > group2_exp)
        less = np.sum(group1_exp < group2_exp)
    
        n1, n2 = len(group1), len(group2)
        delta = (greater - less) / (n1 * n2)

    delta_l.append(delta)

    print('N', n1+n2, n1, n2)

    return (np.round(np.mean(delta_l), 4), np.round(np.std(delta_l)*2.58,100))

def cliff_delta10(group1, group2):

    group1_ = remove_outliers_iqr_10(group1)
    group2_ = remove_outliers_iqr_10(group2)
    
    sub_sample_size = np.min([int(group1_.shape[0]*0.7), int(group2_.shape[0]*0.7)])

    delta_l = []
    for i in range(10):
       
        group1 = resample(group1_, n_samples=sub_sample_size, replace=False, random_state=i)
        group2 = resample(group2_, n_samples=sub_sample_size, replace=False, random_state=i)
    
        group1_exp = group1[:, np.newaxis]  
        group2_exp = group2[np.newaxis, :]  
    
        greater = np.sum(group1_exp > group2_exp)
        less = np.sum(group1_exp < group2_exp)
    
        n1, n2 = len(group1), len(group2)
        delta = (greater - less) / (n1 * n2)

    delta_l.append(delta)

    print('N', n1+n2, n1, n2)

    return (np.round(np.mean(delta_l), 4), np.round(np.std(delta_l)*2.58,100))


def calculate_z_from_u(group1, group2, u_statistic):
    group1 = remove_outliers_iqr(group1)
    group2 = remove_outliers_iqr(group2)


    n1, n2 = len(group1), len(group2)

    mu_u = (n1 * n2) / 2
    sigma_u = np.sqrt((n1 * n2 * (n1 + n2 + 1)) / 12)

    z_value = (u_statistic - mu_u) / sigma_u

    r_value = z_value / np.sqrt(n1 + n2)

    return r_value


def rank_biserial_from_u(group1, group2):
    group1 = remove_outliers_iqr_01(group1)
    group2 = remove_outliers_iqr_01(group2)

    u_statistic, _ = stats.mannwhitneyu(group1, group2, alternative='two-sided')

    n1, n2 = len(group1), len(group2)

    r_biserial = 1 - (2 * u_statistic) / (n1 * n2)

    return r_biserial

def rank_biserial_from_u10(group1, group2):
    group1 = remove_outliers_iqr_10(group1)
    group2 = remove_outliers_iqr_10(group2)

    u_statistic, _ = stats.mannwhitneyu(group1, group2, alternative='two-sided')

    n1, n2 = len(group1), len(group2)

    r_biserial = 1 - (2 * u_statistic) / (n1 * n2)

    return r_biserial

In [6]:
BAGs_data = pd.read_excel('data/BAGs_for_statistical_test.xlsx')


In [7]:

import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt


order = ['Europe', 'Asia', 'LatinAmerica', 'Africa']

regions = BAGs_data['Region1'].unique()
mann_whitney_results = []

for i in range(len(regions)):
    for j in range(i + 1, len(regions)):
        region1_data = BAGs_data[BAGs_data['Region1'] == regions[i]]['GAP_corrected']
        region2_data = BAGs_data[BAGs_data['Region1'] == regions[j]]['GAP_corrected']


        sub_sample_size = np.min([int(region1_data.shape[0]*0.9), int(region2_data.shape[0]*0.9)])
        region1_data = resample(region1_data, n_samples=sub_sample_size, replace=False, random_state=42)
        region2_data = resample(region2_data, n_samples=sub_sample_size, replace=False, random_state=42)

        stat, p_value = stats.mannwhitneyu(region1_data, region2_data, alternative='two-sided')
        
        cliff_d, cliff_ci = cliff_delta(np.array(region1_data), np.array(region2_data))
        print( "cliff delta",cliff_d, cliff_ci)
        
        cliff_d05, cliff_ci05 = cliff_delta05(np.array(region1_data), np.array(region2_data))
        cliff_d10, cliff_ci10 = cliff_delta10(np.array(region1_data), np.array(region2_data))

        r_b = rank_biserial_from_u(np.array(region1_data), np.array(region2_data))
        r_b10 = rank_biserial_from_u10(np.array(region1_data), np.array(region2_data))

        print(f"No parametric cliff_delta: {cliff_d:.3f}")

        p_value_ = permtest(region1_data, region2_data)
        mann_whitney_results.append((regions[i], regions[j], cliff_d, cliff_ci, cliff_d05, cliff_ci05, cliff_d10, r_b10, cliff_ci10, p_value_*len(regions), p_value*len(regions)))


results_list = []
for result in mann_whitney_results:

    result_dict = {
        "Group1": result[0],
        "Group2": result[1],
        "cliff_delta 0.1*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[2]),
        "CI 0.1*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[3]),
        "cliff_delta 0.5*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[4]),
        "CI 0.5*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[5]),
        "cliff_delta 1.0*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[6]),
        "CI 1.0*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[7]),
        "p-value 100000 perm": np.abs(result[8]),
        "p-value mann whitney": np.abs(result[9]),
    }
    results_list.append(result_dict)

results_df = pd.DataFrame(results_list)
display(results_df)


N 12426 6213 6213
cliff delta -0.1995 0.0
N 17054 8527 8527
N 20056 10028 10028
No parametric cliff_delta: -0.200
Computing permutations...
N 200 100 100
cliff delta -0.5596 0.0
N 284 142 142
N 324 162 162
No parametric cliff_delta: -0.560
Computing permutations...
N 12590 6295 6295
cliff delta 0.3093 0.0
N 17658 8829 8829
N 20442 10221 10221
No parametric cliff_delta: 0.309
Computing permutations...
N 202 101 101
cliff delta -0.4216 0.0
N 280 140 140
N 324 162 162
No parametric cliff_delta: -0.422
Computing permutations...
N 42434 21217 21217
cliff delta 0.5408 0.0
N 58440 29220 29220
N 68730 34365 34365
No parametric cliff_delta: 0.541
Computing permutations...
N 202 101 101
cliff delta 0.7263 0.0
N 282 141 141
N 324 162 162
No parametric cliff_delta: 0.726
Computing permutations...


,Group1,Group2,cliff_delta 0.1*IQR (0.15_S | 0.33_M | 0.47_L),CI 0.1*IQR (0.15_S | 0.33_M | 0.47_L),cliff_delta 0.5*IQR (0.15_S | 0.33_M | 0.47_L),CI 0.5*IQR (0.15_S | 0.33_M | 0.47_L),cliff_delta 1.0*IQR (0.15_S | 0.33_M | 0.47_L),CI 1.0*IQR (0.15_S | 0.33_M | 0.47_L),p-value 100000 perm,p-value mann whitney
0,Asia,LatinAmerica,0.1995,0.0,0.1363,0.0,0.0910,0.087436,0.0,0.0
1,Asia,Africa,0.5596,0.0,0.3636,0.0,0.3169,0.297122,0.0,0.0
2,Asia,Europa,0.3093,0.0,0.2241,0.0,0.1857,0.189314,0.0,0.0
3,LatinAmerica,Africa,0.4216,0.0,0.2770,0.0,0.3219,0.260264,0.0,0.0
4,LatinAmerica,Europa,0.5408,0.0,0.3956,0.0,0.2890,0.288315,0.0,0.0
5,Africa,Europa,0.7263,0.0,0.3743,0.0,0.4736,0.413351,0.0,0.0


In [8]:
cliff_ci

0.0

In [9]:
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt


BAGs_data_europe = BAGs_data[BAGs_data['Region1'] == 'Europa']


order = ['WesternEurope', 'EasternEurope', 'NorthernEurope', 'SouthernEurope']


BAGs_data_no_outliers = BAGs_data_europe

order = ['WesternEurope', 'EasternEurope', 'NorthernEurope', 'SouthernEurope']


regions = BAGs_data_no_outliers['Region'].unique()
mann_whitney_results = []

for i in range(len(regions)):
    for j in range(i + 1, len(regions)):
        region1_data = BAGs_data_no_outliers[BAGs_data_no_outliers['Region'] == regions[i]]['GAP_corrected']
        region2_data = BAGs_data_no_outliers[BAGs_data_no_outliers['Region'] == regions[j]]['GAP_corrected']


        sub_sample_size = np.min([int(region1_data.shape[0]*0.9), int(region2_data.shape[0]*0.9)])
        region1_data = resample(region1_data, n_samples=sub_sample_size, replace=False, random_state=42)
        region2_data = resample(region2_data, n_samples=sub_sample_size, replace=False, random_state=42)
        
        stat, p_value = stats.mannwhitneyu(region1_data, region2_data, alternative='two-sided')
        
        cliff_d, cliff_ci = cliff_delta(np.array(region1_data), np.array(region2_data))
        print( "cliff delta",cliff_d, cliff_ci)
        
        cliff_d05, cliff_ci05 = cliff_delta05(np.array(region1_data), np.array(region2_data))
        cliff_d10, cliff_ci10 = cliff_delta10(np.array(region1_data), np.array(region2_data))

        r_b = rank_biserial_from_u(np.array(region1_data), np.array(region2_data))
        r_b10 = rank_biserial_from_u10(np.array(region1_data), np.array(region2_data))

        print(f"No parametric cliff_delta: {cliff_d:.3f}")
        print(f"No parametric biseral: {r_b:.3f}")

        p_value_ = permtest(region1_data, region2_data)
        mann_whitney_results.append((regions[i], regions[j], cliff_d, cliff_ci, cliff_d05, cliff_ci05, cliff_d10, r_b10, cliff_ci10, p_value_*len(regions), p_value*len(regions)))


results_list = []
for result in mann_whitney_results:

    result_dict = {
        "Group1": result[0],
        "Group2": result[1],
        "cliff_delta 0.1*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[2]),
        "CI 0.1*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[3]),
        "cliff_delta 0.5*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[4]),
        "CI 0.5*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[5]),
        "cliff_delta 1.0*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[6]),
        "CI 1.0*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[7]),
        "p-value 100000 perm": np.abs(result[8]),
        "p-value mann whitney": np.abs(result[9]),
    }
    results_list.append(result_dict)

results_df = pd.DataFrame(results_list)
display(results_df)


N 17994 8997 8997
cliff delta -0.1363 0.0
N 25234 12617 12617
N 29440 14720 14720
No parametric cliff_delta: -0.136
No parametric biseral: 0.128
Computing permutations...
N 13486 6743 6743
cliff delta -0.0454 0.0
N 18966 9483 9483
N 22150 11075 11075
No parametric cliff_delta: -0.045
No parametric biseral: 0.038
Computing permutations...
N 11896 5948 5948
cliff delta -0.2257 0.0
N 16730 8365 8365
N 19532 9766 9766
No parametric cliff_delta: -0.226
No parametric biseral: 0.233
Computing permutations...
N 13640 6820 6820
cliff delta 0.0947 0.0
N 19380 9690 9690
N 22370 11185 11185
No parametric cliff_delta: 0.095
No parametric biseral: -0.094
Computing permutations...
N 12172 6086 6086
cliff delta -0.1001 0.0
N 17242 8621 8621
N 19774 9887 9887
No parametric cliff_delta: -0.100
No parametric biseral: 0.102
Computing permutations...
N 12026 6013 6013
cliff delta -0.1838 0.0
N 17052 8526 8526
N 19696 9848 9848
No parametric cliff_delta: -0.184
No parametric biseral: 0.194
Computing permuta

,Group1,Group2,cliff_delta 0.1*IQR (0.15_S | 0.33_M | 0.47_L),CI 0.1*IQR (0.15_S | 0.33_M | 0.47_L),cliff_delta 0.5*IQR (0.15_S | 0.33_M | 0.47_L),CI 0.5*IQR (0.15_S | 0.33_M | 0.47_L),cliff_delta 1.0*IQR (0.15_S | 0.33_M | 0.47_L),CI 1.0*IQR (0.15_S | 0.33_M | 0.47_L),p-value 100000 perm,p-value mann whitney
0,WesternEurope,EasternEurope,0.1363,0.0,0.0769,0.0,0.0709,0.072300,0.0,0.00000
1,WesternEurope,NorthernEurope,0.0454,0.0,0.0202,0.0,0.0171,0.018047,0.0,0.00536
2,WesternEurope,SouthernEurope,0.2257,0.0,0.1425,0.0,0.1369,0.137311,0.0,0.00000
3,EasternEurope,NorthernEurope,0.0947,0.0,0.0551,0.0,0.0548,0.053918,0.0,0.00000
4,EasternEurope,SouthernEurope,0.1001,0.0,0.0712,0.0,0.0600,0.065898,0.0,0.00000
5,NorthernEurope,SouthernEurope,0.1838,0.0,0.1195,0.0,0.1202,0.118086,0.0,0.00000


In [10]:
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt


q1 = BAGs_data['GAP_corrected'].quantile(0.25)
q3 = BAGs_data['GAP_corrected'].quantile(0.75)
iqr = q3 - q1

lower_bound = q1 - 1.0 * iqr
upper_bound = q3 + 1.0 * iqr


BAGs_data_no_outliers = BAGs_data

bins = [0, BAGs_data['GDP_percapita'].median(), float('inf')]  # Umbrales en dólares
labels = ['L_GDP', 'H_GDP']


BAGs_data_no_outliers['Group_GDP'] = pd.cut(BAGs_data_no_outliers['GDP_percapita'], bins=bins, labels=labels)

order = ['L_GDP', 'H_GDP']

regions = ['L_GDP', 'H_GDP']
mann_whitney_results = []

for i in range(len(regions)):
    for j in range(i + 1, len(regions)):
        region1_data = BAGs_data_no_outliers[BAGs_data_no_outliers['Group_GDP'] == regions[i]]['GAP_corrected']
        region2_data = BAGs_data_no_outliers[BAGs_data_no_outliers['Group_GDP'] == regions[j]]['GAP_corrected']


        sub_sample_size = np.min([int(region1_data.shape[0]*0.9), int(region2_data.shape[0]*0.9)])
        region1_data = resample(region1_data, n_samples=sub_sample_size, replace=False, random_state=42)
        region2_data = resample(region2_data, n_samples=sub_sample_size, replace=False, random_state=42)
        
        stat, p_value = stats.mannwhitneyu(region1_data, region2_data, alternative='two-sided')
        
        cliff_d, cliff_ci = cliff_delta(np.array(region1_data), np.array(region2_data))
        print( "cliff delta",cliff_d, cliff_ci)
        
        cliff_d05, cliff_ci05 = cliff_delta05(np.array(region1_data), np.array(region2_data))
        cliff_d10, cliff_ci10 = cliff_delta10(np.array(region1_data), np.array(region2_data))

        r_b = rank_biserial_from_u(np.array(region1_data), np.array(region2_data))
        r_b10 = rank_biserial_from_u10(np.array(region1_data), np.array(region2_data))

        print(f"No parametric cliff_delta: {cliff_d:.3f}")
        print(f"No parametric biseral: {r_b:.3f}")

        p_value_ = permtest(region1_data, region2_data)
        mann_whitney_results.append((regions[i], regions[j], cliff_d, cliff_ci, cliff_d05, cliff_ci05, cliff_d10, r_b10, cliff_ci10, p_value_*len(regions), p_value*len(regions)))


results_list = []
for result in mann_whitney_results:

    result_dict = {
        "Group1": result[0],
        "Group2": result[1],
        "cliff_delta 0.1*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[2]),
        "CI 0.1*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[3]),
        "cliff_delta 0.5*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[4]),
        "CI 0.5*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[5]),
        "cliff_delta 1.0*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[6]),
        "CI 1.0*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[7]),
        "p-value 100000 perm": np.abs(result[8]),
        "p-value mann whitney": np.abs(result[9]),
    }
    results_list.append(result_dict)

results_df = pd.DataFrame(results_list)
display(results_df)


N 51748 25874 25874
cliff delta 0.5687 0.0
N 72264 36132 36132
N 84984 42492 42492
No parametric cliff_delta: 0.569
No parametric biseral: -0.571
Computing permutations...


,Group1,Group2,cliff_delta 0.1*IQR (0.15_S | 0.33_M | 0.47_L),CI 0.1*IQR (0.15_S | 0.33_M | 0.47_L),cliff_delta 0.5*IQR (0.15_S | 0.33_M | 0.47_L),CI 0.5*IQR (0.15_S | 0.33_M | 0.47_L),cliff_delta 1.0*IQR (0.15_S | 0.33_M | 0.47_L),CI 1.0*IQR (0.15_S | 0.33_M | 0.47_L),p-value 100000 perm,p-value mann whitney
0,L_GDP,H_GDP,0.5687,0.0,0.4096,0.0,0.301,0.300884,0.0,0.0


In [11]:
BAGs_data['GDP_percapita'].median()

17402.03761280788

In [13]:
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt


q1 = BAGs_data['GAP_corrected'].quantile(0.25)
q3 = BAGs_data['GAP_corrected'].quantile(0.75)
iqr = q3 - q1

lower_bound = q1 - 1.0 * iqr
upper_bound = q3 + 1.0 * iqr

BAGs_data_no_outliers = BAGs_data

order = ['L_Ineq', 'H_Ineq']

regions = ['L_Ineq', 'H_Ineq']
mann_whitney_results = []

for i in range(len(regions)):
    for j in range(i + 1, len(regions)):
        region1_data = BAGs_data_no_outliers[BAGs_data_no_outliers['Group_GINI'] == regions[i]]['GAP_corrected']
        region2_data = BAGs_data_no_outliers[BAGs_data_no_outliers['Group_GINI'] == regions[j]]['GAP_corrected']


        sub_sample_size = np.min([int(region1_data.shape[0]*0.9), int(region2_data.shape[0]*0.9)])
        region1_data = resample(region1_data, n_samples=sub_sample_size, replace=False, random_state=42)
        region2_data = resample(region2_data, n_samples=sub_sample_size, replace=False, random_state=42)
        
        stat, p_value = stats.mannwhitneyu(region1_data, region2_data, alternative='two-sided')
        
        cliff_d, cliff_ci = cliff_delta(np.array(region1_data), np.array(region2_data))
        print( "cliff delta",cliff_d, cliff_ci)
        
        cliff_d05, cliff_ci05 = cliff_delta05(np.array(region1_data), np.array(region2_data))
        cliff_d10, cliff_ci10 = cliff_delta10(np.array(region1_data), np.array(region2_data))

        print(f"No parametric cliff_delta: {cliff_d:.3f}")

        p_value_ = permtest(region1_data, region2_data)
        mann_whitney_results.append((regions[i], regions[j], cliff_d, cliff_ci, cliff_d05, cliff_ci05, cliff_d10, r_b10, cliff_ci10, p_value_*len(regions), p_value*len(regions)))


results_list = []
for result in mann_whitney_results:

    result_dict = {
        "Group1": result[0],
        "Group2": result[1],
        "cliff_delta 0.1*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[2]),
        "CI 0.1*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[3]),
        "cliff_delta 0.5*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[4]),
        "CI 0.5*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[5]),
        "cliff_delta 1.0*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[6]),
        "CI 1.0*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[7]),
        "p-value 100000 perm": np.abs(result[8]),
        "p-value mann whitney": np.abs(result[9]),
    }
    results_list.append(result_dict)

results_df = pd.DataFrame(results_list)
display(results_df)


N 57726 28863 28863
cliff delta -0.5728 0.0
N 80670 40335 40335
N 94490 47245 47245
No parametric cliff_delta: -0.573
Computing permutations...


,Group1,Group2,cliff_delta 0.1*IQR (0.15_S | 0.33_M | 0.47_L),CI 0.1*IQR (0.15_S | 0.33_M | 0.47_L),cliff_delta 0.5*IQR (0.15_S | 0.33_M | 0.47_L),CI 0.5*IQR (0.15_S | 0.33_M | 0.47_L),cliff_delta 1.0*IQR (0.15_S | 0.33_M | 0.47_L),CI 1.0*IQR (0.15_S | 0.33_M | 0.47_L),p-value 100000 perm,p-value mann whitney
0,L_Ineq,H_Ineq,0.5728,0.0,0.4054,0.0,0.3055,0.300884,0.0,0.0


In [14]:
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt



q1 = BAGs_data['GAP_corrected'].quantile(0.25)
q3 = BAGs_data['GAP_corrected'].quantile(0.75)
iqr = q3 - q1

lower_bound = q1 - 1.0 * iqr
upper_bound = q3 + 1.0 * iqr

BAGs_data_no_outliers = BAGs_data

BAGs_data_no_outliers.loc[BAGs_data_no_outliers['country'] == 'Colombia', 'Income'] = 'L'
BAGs_data_no_outliers.loc[BAGs_data_no_outliers['country'] == 'Ecuador', 'Income'] = 'L'
BAGs_data_no_outliers.loc[BAGs_data_no_outliers['country'] == 'Brasil', 'Income'] = 'L'
BAGs_data_no_outliers.loc[BAGs_data_no_outliers['country'] == 'Italy', 'Income'] = 'H'


BAGs_data_no_outliers['Income_v1'] = BAGs_data_no_outliers['Income'] 

BAGs_data_no_outliers['Income_v1'] = BAGs_data_no_outliers['Income_v1'].replace({'UM':'H'})

order = ['H', 'L']

regions = ['H', 'L']
mann_whitney_results = []


    
for i in range(len(regions)):
    for j in range(i + 1, len(regions)):
        region1_data = BAGs_data[BAGs_data['Income_v1'] == regions[i]]['GAP_corrected']
        region2_data = BAGs_data[BAGs_data['Income_v1'] == regions[j]]['GAP_corrected']



        sub_sample_size = np.min([int(region1_data.shape[0]*0.9), int(region2_data.shape[0]*0.9)])
        region1_data = resample(region1_data, n_samples=sub_sample_size, replace=False, random_state=42)
        region2_data = resample(region2_data, n_samples=sub_sample_size, replace=False, random_state=42)
    
        stat, p_value = stats.mannwhitneyu(region1_data, region2_data, alternative='two-sided')
        
        cliff_d, cliff_ci = cliff_delta(np.array(region1_data), np.array(region2_data))
        print( "cliff delta",cliff_d, cliff_ci)
        
        cliff_d05, cliff_ci05 = cliff_delta05(np.array(region1_data), np.array(region2_data))
        cliff_d10, cliff_ci10 = cliff_delta10(np.array(region1_data), np.array(region2_data))

        r_b = rank_biserial_from_u(np.array(region1_data), np.array(region2_data))
        r_b10 = rank_biserial_from_u10(np.array(region1_data), np.array(region2_data))

        print(f"No parametric cliff_delta: {cliff_d:.3f}")
        print(f"No parametric biseral: {r_b:.3f}")

        p_value_ = permtest(region1_data, region2_data)
        mann_whitney_results.append((regions[i], regions[j], cliff_d, cliff_ci, cliff_d05, cliff_ci05, cliff_d10, r_b10, cliff_ci10, p_value_*len(regions), p_value*len(regions)))


results_list = []
for result in mann_whitney_results:

    result_dict = {
        "Group1": result[0],
        "Group2": result[1],
        "cliff_delta 0.1*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[2]),
        "CI 0.1*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[3]),
        "cliff_delta 0.5*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[4]),
        "CI 0.5*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[5]),
        "cliff_delta 1.0*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[6]),
        "CI 1.0*IQR (0.15_S | 0.33_M | 0.47_L)": np.abs(result[7]),
        "p-value 100000 perm": np.abs(result[8]),
        "p-value mann whitney": np.abs(result[9]),
    }
    results_list.append(result_dict)
    
results_df = pd.DataFrame(results_list)
display(results_df)



N 26168 13084 13084
cliff delta -0.4897 0.0
N 35638 17819 17819
N 41572 20786 20786
No parametric cliff_delta: -0.490
No parametric biseral: 0.493
Computing permutations...


,Group1,Group2,cliff_delta 0.1*IQR (0.15_S | 0.33_M | 0.47_L),CI 0.1*IQR (0.15_S | 0.33_M | 0.47_L),cliff_delta 0.5*IQR (0.15_S | 0.33_M | 0.47_L),CI 0.5*IQR (0.15_S | 0.33_M | 0.47_L),cliff_delta 1.0*IQR (0.15_S | 0.33_M | 0.47_L),CI 1.0*IQR (0.15_S | 0.33_M | 0.47_L),p-value 100000 perm,p-value mann whitney
0,H,L,0.4897,0.0,0.3452,0.0,0.2619,0.260627,0.0,0.0
